# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121375e+02     1.562694e+00
 * time: 0.0894169807434082
     1     1.074296e+01     8.988096e-01
 * time: 0.905264139175415
     2    -1.237579e+01     1.111649e+00
 * time: 1.0850131511688232
     3    -3.396068e+01     7.799115e-01
 * time: 1.2763769626617432
     4    -4.760119e+01     5.900584e-01
 * time: 1.4172930717468262
     5    -5.690112e+01     2.013989e-01
 * time: 1.5708460807800293
     6    -5.968903e+01     1.723497e-01
 * time: 1.6880919933319092
     7    -6.085732e+01     7.627683e-02
 * time: 1.7971031665802002
     8    -6.138053e+01     4.096555e-02
 * time: 1.9146971702575684
     9    -6.168482e+01     3.375647e-02
 * time: 2.0231809616088867
    10    -6.185498e+01     2.646100e-02
 * time: 2.150473117828369
    11    -6.196582e+01     2.568325e-02
 * time: 2.267524003982544
    12    -6.204326e+01     1.664407e-02
 * time: 2.376829147338867
    13    -6.210017e+01     1.368113e-02
 * time: 2.49482512474060

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671052
    AtomicLocal         -18.8557754
    AtomicNonlocal      14.8522737
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336817

    total               -62.261666453800
